# Calcul du taux d'émission pour les véhicules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns

In [2]:
essence=pd.read_csv("emissions_essence.csv")
diesel=pd.read_csv("emissions_diesel.csv")
euro=pd.read_csv("norme_euro.csv")
data=pd.read_csv("Donnees-sur-le-parc-de-vehicule-au-niveau-regional.2023-05.csv", sep=';',header=[1])
data = data.loc[data["CATEGORIE_VEHICULE"] == "Véhicule particulier"]
data.reset_index(drop=True, inplace=True)

In [3]:
essence

,Norme,Euro 1,Euro 2,Euro 3,Euro 4,Euro 5[25],Euro 6b[26],Euro 6c,Euro 6d -TEMP,Euro 6d
0,Oxydes d'azote (NOx),-,-,150,80,60,60,60,60,60
1,Monoxyde de carbone (CO),2 720,2 200,2 200,1 000,1 000,1 000,1 000,1 000,1 000
2,Hydrocarbures (HC),-,-,200,100,100,100,100,100,100
3,Hydrocarbures non méthaniques (HCNM),-,-,-,-,68,68,68,68,68
4,HC + NOX,970,500,-,-,-,-,-,-,-
5,Particules (PM),-,-,-,-,5[* 1],5[* 1],45,45,45
6,Particules (PN) (nb/km),-,-,-,-,-,6 × 1012[* 2],6 × 1011,6 × 1011,6 × 1011


In [4]:
diesel

,Norme,Euro 1,Euro 2,Euro 3,Euro 4,Euro 5a,Euro 5b,Euro 6b,Euro 6c,Euro 6d -TEMP,Euro 6d
0,Oxydes d'azote (NOx),-,-,500,250,180,180,80,80,80,80
1,Monoxyde de carbone (CO),2 720,1 000,640,500,500,500,500,500,500,500
2,Hydrocarbures (THC),-,-,-,-,-,-,-,-,-,-
3,Hydrocarbures non méthaniques (HCNM),-,-,-,-,-,-,-,-,-,-
4,HC + NOX,970,700,560,300,230,230,170,170,170,170
5,Particules (PM),140,80,50,25,5,45,45,45,45,45
6,Particules (PN) (nb/km),-,-,-,-,6 × 1011,6 × 1011,6 × 1011,6 × 1011,6 × 1011,6 × 1011


In [5]:
#euro=euro.drop(2)
euro

,Norme,Mise en service des véhicules
0,Euro 1,1er janvier 1993
1,Euro 2 (voitures),1er janvier 1997
2,Euro 2 (Véhicules utilitaires légers),1er octobre 1997
3,Euro 3,1er janvier 2001
4,Euro 4,1er janvier 2006
5,Euro 5a,1er janvier 2011
6,Euro 5b,1er janvier 2013
7,Euro 6b,1er septembre 2015
8,Euro 6c,1er septembre 2018
9,Euro 6d-TEMP[8],1er septembre 2019


Il sera facile de donner la norme pour tout les autres véhicules entrant à partir de 2021 puisqu'elle sera "euro 6d". Nous allons ici donner le nombre de véhicule pour chaque norme de 2011 à 2020.

In [6]:
variables = ['PARC_2011','PARC_2012','PARC_2013','PARC_2014','PARC_2015','PARC_2016','PARC_2017','PARC_2018','PARC_2019','PARC_2020','PARC_2021','PARC_2022']

In [7]:
df1 = data.copy()

def age_num(age) :
    res = []
    for i in range (len(age)) :
        res.append(int(str(age[i])[0:2]))
    return res

# On créé une colonne avec l'âge du véhicule sous forme numérique
df1.insert(loc = 1, column = 'AGE_NUM', value = age_num(np.array(df1['AGE'], dtype = str)))

# On peut se séparer de certaines colonnes
df1 = df1.drop(['REGION_CODE', 'REGION_LIBELLE', 'CLASSE_VEHICULE', 'STATUT_UTILISATEUR', 'AGE'], axis = 1)
df1=df1.loc[(df1["CARBURANT"] == "Diesel") |
                (df1["CARBURANT"] == "Essence")] # on ne garde que les carburants diesel et essence
            
# On effectue la somme pour les différentes régions et statuts d'utilisateurs
df1 = df1.groupby(['AGE_NUM','CARBURANT'])[variables].sum()
df1 = df1.sort_values(by = ['AGE_NUM'])

In [8]:
#df1

In [9]:
df_essence = df1.iloc[1::2]
df_diesel = df1.iloc[::2]

In [10]:
nb_diesel=df_diesel.reset_index().drop(['CARBURANT','AGE_NUM'] ,axis=1)
nb_essence=df_essence.reset_index().drop(['CARBURANT','AGE_NUM'] ,axis=1)

In [11]:
nb_diesel

,PARC_2011,PARC_2012,PARC_2013,PARC_2014,PARC_2015,PARC_2016,PARC_2017,PARC_2018,PARC_2019,PARC_2020,PARC_2021,PARC_2022
0,1599354,1592652,1388397,1216709,1169419,1112160,1059582,993582,846464,759842,527533,366426
1,1680776,1657012,1666343,1466534,1286505,1214888,1145761,1086385,1020140,882019,796809,560365
2,1754070,1698877,1683979,1699549,1493332,1299749,1228282,1156457,1112097,1060225,900605,819308
3,1705835,1741761,1682757,1668758,1676633,1470484,1277932,1194649,1133356,1099350,1047686,890775
4,1503173,1623027,1662775,1617713,1603723,1603135,1401721,1209765,1128752,1081787,1046921,982328
5,1471190,1493374,1615599,1660996,1610790,1596440,1594372,1388582,1209137,1139470,1092552,1060269
6,1407227,1441662,1468451,1601500,1652364,1601986,1584028,1576457,1372900,1211327,1139974,1085810
7,1347362,1388526,1426066,1460444,1597825,1643383,1588044,1565534,1560691,1369152,1205848,1131755
8,1292891,1314513,1358142,1399914,1441756,1572652,1611274,1552167,1523755,1537409,1351282,1178892
9,1188381,1258310,1281233,1327128,1372252,1413939,1537554,1569996,1511694,1495976,1509790,1322372


In [12]:
def DateMiseCirculation(data):
    df=data.copy()
    df=pd.DataFrame(df)
    age=np.arange(0,26,1)
    i=0
    for k in range (2011,2023):
        annee_mise_service=[k - nombre for nombre in age]
        df.iloc[:,i]=annee_mise_service
        i+=1
    return df
        

In [13]:
essence=DateMiseCirculation(nb_essence)
diesel=DateMiseCirculation(nb_diesel)

In [23]:
essence

,PARC_2011,PARC_2012,PARC_2013,PARC_2014,PARC_2015,PARC_2016,PARC_2017,PARC_2018,PARC_2019,PARC_2020,PARC_2021,PARC_2022
0,Euro 5a,Euro 5a,Euro 5b,Euro 5b,Euro 6b,Euro 6b,Euro 6b,Euro 6c,Euro 6d-TEMP[8],Euro 6d-TEMP[8],Euro 6d[9],Euro 6d[9]
1,Euro 4,Euro 5a,Euro 5a,Euro 5b,Euro 5b,Euro 6b,Euro 6b,Euro 6b,Euro 6c,Euro 6d-TEMP[8],Euro 6d-TEMP[8],Euro 6d[9]
2,Euro 4,Euro 4,Euro 5a,Euro 5a,Euro 5b,Euro 5b,Euro 6b,Euro 6b,Euro 6b,Euro 6c,Euro 6d-TEMP[8],Euro 6d-TEMP[8]
3,Euro 4,Euro 4,Euro 4,Euro 5a,Euro 5a,Euro 5b,Euro 5b,Euro 6b,Euro 6b,Euro 6b,Euro 6c,Euro 6d-TEMP[8]
4,Euro 4,Euro 4,Euro 4,Euro 4,Euro 5a,Euro 5a,Euro 5b,Euro 5b,Euro 6b,Euro 6b,Euro 6b,Euro 6c
5,Euro 4,Euro 4,Euro 4,Euro 4,Euro 4,Euro 5a,Euro 5a,Euro 5b,Euro 5b,Euro 6b,Euro 6b,Euro 6b
6,Euro 3,Euro 4,Euro 4,Euro 4,Euro 4,Euro 4,Euro 5a,Euro 5a,Euro 5b,Euro 5b,Euro 6b,Euro 6b
7,Euro 3,Euro 3,Euro 4,Euro 4,Euro 4,Euro 4,Euro 4,Euro 5a,Euro 5a,Euro 5b,Euro 5b,Euro 6b
8,Euro 3,Euro 3,Euro 3,Euro 4,Euro 4,Euro 4,Euro 4,Euro 4,Euro 5a,Euro 5a,Euro 5b,Euro 5b
9,Euro 3,Euro 3,Euro 3,Euro 3,Euro 4,Euro 4,Euro 4,Euro 4,Euro 4,Euro 5a,Euro 5a,Euro 5b


In [15]:
def attribution_euro(date):
    if date < 1987:
        return 'Euro 1'
    elif date >=1987 and date < 2001:
        return 'Euro 2 (voitures)'
    elif date >=2001 and date < 2006:
        return 'Euro 3'
    elif date >=2006 and date < 2011:
        return 'Euro 4'
    elif date >=2011 and date < 2013:
        return 'Euro 5a'
    elif date >=2013 and date < 2015:
        return 'Euro 5b'
    elif date >=2015 and date < 2018:
        return 'Euro 6b'
    elif date >=2018 and date < 2019:
        return 'Euro 6c'
    elif date >=2019 and date < 2021:
        return 'Euro 6d-TEMP[8]'
    elif date >=2021:
        return 'Euro 6d[9]'
        

In [16]:
essence=essence.applymap(attribution_euro)
diesel=diesel.applymap(attribution_euro)

/tmp/ipykernel_2236/3994447616.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  essence=essence.applymap(attribution_euro)
/tmp/ipykernel_2236/3994447616.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  diesel=diesel.applymap(attribution_euro)


In [24]:
diesel.groupby(euro['Norme'])